In [5]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re
from datetime import datetime
import locale

In [6]:
locale.setlocale(locale.LC_ALL, 'pt_BR')

'pt_BR'

## Aposentadoria gráfico 2

In [7]:
client = bigquery.Client()

In [8]:
query = """ 
        SELECT ANO as ano, MES as mes,  Id_SERVIDOR_PORTAL as id_servidor, TIPO_APOSENTADORIA as tipo_aposentadoria, DATA_APOSENTADORIA as data_aposentadoria, DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO as data_diploma_ingresso_servicopublico,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_nov_siape_aposentados_cadastro`
        """  

In [9]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,id_servidor,tipo_aposentadoria,data_aposentadoria,data_diploma_ingresso_servicopublico,org_lotacao,regime_juridico,total
0,2023,11,311786085,APOSENTADORIA POR INVALIDEZ,30/04/2008,12/08/1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
1,2023,11,311413108,APOSENTADORIA VOLUNTARIA,07/04/2017,01/08/1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1
2,2023,11,310796209,APOSENTADORIA VOLUNTARIA,12/04/2002,15/03/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
3,2023,11,313314589,APOSENTADORIA VOLUNTARIA,12/05/2011,01/10/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
4,2023,11,312332839,APOSENTADORIA VOLUNTARIA,14/06/2002,15/02/1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
418496,2023,11,312385265,APOSENTADORIA VOLUNTARIA,30/09/2009,01/09/1979,Ministério da Defesa,REGIME JURIDICO UNICO,1
418497,2023,11,312133569,APOSENTADORIA VOLUNTARIA,02/04/2012,17/09/1980,Ministério da Defesa,REGIME JURIDICO UNICO,1
418498,2023,11,312305623,APOSENTADORIA VOLUNTARIA,06/09/1994,01/04/1984,Ministério da Defesa,REGIME JURIDICO UNICO,1
418499,2023,11,311654066,APOSENTADORIA POR INVALIDEZ,15/09/2009,01/09/1977,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [10]:
df.columns

Index(['ano', 'mes', 'id_servidor', 'tipo_aposentadoria', 'data_aposentadoria',
       'data_diploma_ingresso_servicopublico', 'org_lotacao',
       'regime_juridico', 'total'],
      dtype='object')

In [11]:
df.dtypes

ano                                      Int64
mes                                      Int64
id_servidor                              Int64
tipo_aposentadoria                      object
data_aposentadoria                      object
data_diploma_ingresso_servicopublico    object
org_lotacao                             object
regime_juridico                         object
total                                    Int64
dtype: object

In [12]:
aposentadoria = df['data_aposentadoria'].isnull()
aposentadoria.value_counts()

False    417698
True        803
Name: data_aposentadoria, dtype: int64

In [13]:
ingresso = df['data_diploma_ingresso_servicopublico'].isnull()
ingresso.value_counts()

False    412512
True       5989
Name: data_diploma_ingresso_servicopublico, dtype: int64

In [14]:
df = df.dropna() 
df #Dados nulos da data de aposentadoria = 803; Dados nulos da data de ingresso = 5989. A soma dos dois é a quantidade de linhas dropadas (6792). 


,ano,mes,id_servidor,tipo_aposentadoria,data_aposentadoria,data_diploma_ingresso_servicopublico,org_lotacao,regime_juridico,total
0,2023,11,311786085,APOSENTADORIA POR INVALIDEZ,30/04/2008,12/08/1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
1,2023,11,311413108,APOSENTADORIA VOLUNTARIA,07/04/2017,01/08/1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1
2,2023,11,310796209,APOSENTADORIA VOLUNTARIA,12/04/2002,15/03/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
3,2023,11,313314589,APOSENTADORIA VOLUNTARIA,12/05/2011,01/10/1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
4,2023,11,312332839,APOSENTADORIA VOLUNTARIA,14/06/2002,15/02/1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...
418496,2023,11,312385265,APOSENTADORIA VOLUNTARIA,30/09/2009,01/09/1979,Ministério da Defesa,REGIME JURIDICO UNICO,1
418497,2023,11,312133569,APOSENTADORIA VOLUNTARIA,02/04/2012,17/09/1980,Ministério da Defesa,REGIME JURIDICO UNICO,1
418498,2023,11,312305623,APOSENTADORIA VOLUNTARIA,06/09/1994,01/04/1984,Ministério da Defesa,REGIME JURIDICO UNICO,1
418499,2023,11,311654066,APOSENTADORIA POR INVALIDEZ,15/09/2009,01/09/1977,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [15]:
df['data_aposentadoria'] = pd.to_datetime(df['data_aposentadoria'], dayfirst=True, format= "%d/%m/%Y") #transformando em data

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\1474179940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_aposentadoria'] = pd.to_datetime(df['data_aposentadoria'], dayfirst=True, format= "%d/%m/%Y") #transformando em data


In [16]:
df['data_diploma_ingresso_servicopublico'] = pd.to_datetime(df['data_diploma_ingresso_servicopublico'], dayfirst=True, format= "%d/%m/%Y") #transformando em data

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\3069410005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_diploma_ingresso_servicopublico'] = pd.to_datetime(df['data_diploma_ingresso_servicopublico'], dayfirst=True, format= "%d/%m/%Y") #transformando em data


In [17]:
df['ano_aposentadoria'] = df['data_aposentadoria'].dt.year #pegando apenas o ano

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\1637080438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_aposentadoria'] = df['data_aposentadoria'].dt.year #pegando apenas o ano


In [18]:
df['ano_ingresso'] = df['data_diploma_ingresso_servicopublico'].dt.year #pegando apenas o ano

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\2359990388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_ingresso'] = df['data_diploma_ingresso_servicopublico'].dt.year #pegando apenas o ano


In [19]:
df['ano_ingresso']= df['ano_ingresso'].astype(int) #transformando os valores da coluna em inteiro

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\3392378587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_ingresso']= df['ano_ingresso'].astype(int) #transformando os valores da coluna em inteiro


In [20]:
df['ano_ingresso'].unique()

array([1982, 1981, 1990, 1977, 1985, 1978, 1971, 1986, 1984, 1976, 1980,
       1974, 1972, 1989, 2013, 2014, 1979, 1983, 1987, 1991, 1963, 1954,
       1942, 1949, 1953, 1952, 1992, 2016, 1975, 1988, 1955, 1957, 1950,
       1960, 1956, 1947, 1945, 1959, 1967, 1966, 1965, 1970, 1996, 1995,
       1993, 2008, 1997, 1961, 1968, 1962, 1999, 1998, 1973, 1994, 1958,
       1964, 1969, 1946, 2004, 2006, 2007, 2009, 2011, 2012, 1943, 2010,
       1951, 2000, 1944, 2002, 2001, 2005, 2003, 1948, 1941, 2017, 2019,
       2015, 1939, 1940, 2018, 2020, 2021, 1936, 1932, 1935, 1938])

In [21]:
df['ano_aposentadoria'] = df['ano_aposentadoria'].astype(int) #transformando os valores da coluna em inteiro

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\3254720995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ano_aposentadoria'] = df['ano_aposentadoria'].astype(int) #transformando os valores da coluna em inteiro


In [22]:
df['ano_aposentadoria'].unique()

array([2008, 2017, 2002, 2011, 2000, 2012, 2005, 2004, 2003, 2014, 1988,
       2016, 1991, 2010, 2021, 2018, 1997, 2013, 2015, 1996, 2009, 2019,
       2023, 2022, 2020, 1999, 1987, 1994, 2001, 1992, 2006, 1995, 1986,
       1998, 1977, 1990, 1993, 2007, 1980, 1989, 1985, 1984, 1983, 1981,
       1982, 1979, 1976, 1978, 1965, 1968, 1972, 1971, 1970, 1966, 1967,
       1963, 1974, 1969, 1975, 1973, 1964, 1959, 1962, 1960, 1952, 1953,
       1955, 1958, 1954, 1951, 1961, 1957])

In [23]:
limites = [0, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2024]
categorias = [1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]

df['decada_aposentadoria'] = pd.cut(df['ano_aposentadoria'], bins=limites, labels=categorias)


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\2946031983.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decada_aposentadoria'] = pd.cut(df['ano_aposentadoria'], bins=limites, labels=categorias)


In [24]:
df = df[['ano', 'mes', 'tipo_aposentadoria', 'data_aposentadoria', 'ano_aposentadoria', 'decada_aposentadoria', 'data_diploma_ingresso_servicopublico', 'ano_ingresso', 'org_lotacao', 'regime_juridico', 'total']]
df #411709 linhas

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total
0,2023,11,APOSENTADORIA POR INVALIDEZ,2008-04-30,2008,2000,1982-08-12,1982,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
1,2023,11,APOSENTADORIA VOLUNTARIA,2017-04-07,2017,2010,1981-08-01,1981,MINISTERIO DA CULTURA,REGIME JURIDICO UNICO,1
2,2023,11,APOSENTADORIA VOLUNTARIA,2002-04-12,2002,2000,1990-03-15,1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
3,2023,11,APOSENTADORIA VOLUNTARIA,2011-05-12,2011,2010,1990-10-01,1990,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
4,2023,11,APOSENTADORIA VOLUNTARIA,2002-06-14,2002,2000,1977-02-15,1977,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...,...,...,...,...,...
418496,2023,11,APOSENTADORIA VOLUNTARIA,2009-09-30,2009,2000,1979-09-01,1979,Ministério da Defesa,REGIME JURIDICO UNICO,1
418497,2023,11,APOSENTADORIA VOLUNTARIA,2012-04-02,2012,2010,1980-09-17,1980,Ministério da Defesa,REGIME JURIDICO UNICO,1
418498,2023,11,APOSENTADORIA VOLUNTARIA,1994-09-06,1994,1990,1984-04-01,1984,Ministério da Defesa,REGIME JURIDICO UNICO,1
418499,2023,11,APOSENTADORIA POR INVALIDEZ,2009-09-15,2009,2000,1977-09-01,1977,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [25]:
df['tempo_contribuicao_servico_publico'] = df['ano_aposentadoria'] - df['ano_ingresso']
df['tempo_contribuicao_servico_publico']

0         26
1         36
2         12
3         21
4         25
          ..
418496    30
418497    32
418498    10
418499    32
418500    18
Name: tempo_contribuicao_servico_publico, Length: 411709, dtype: int32

In [26]:
df['tempo_contribuicao_servico_publico'].unique()

array([ 26,  36,  12,  21,  25,  15,  22,  41,  19,  18,  24,  32,  34,
        11,  30,  38,  17,  20,  31,  40,  28,   0,  23,  27,   2,  14,
        33,  16,  39,  29,  46,  55,  67,  60,  48,  56,  57,  42,   5,
        37,  43,  13,  44,  45,   1,   7,   3,   4,   8,  35,  50,  47,
        54,  49,   6,  52,  10,  -8,   9,  53,  66,  -5, -27,  -1,  -2,
       -12,  -3,  58, -13, -14,  51,  -6, -16, -21,  -7,  59,  -9,  -4,
        61,  65,  62, -17,  64,  63, -24, -11, -10, -15, -18, -19, -22])

In [27]:
df_zeros = df[df['tempo_contribuicao_servico_publico']==0] #1338 linhas
df_zeros

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total,tempo_contribuicao_servico_publico
28,2023,11,APOSENTADORIA VOLUNTARIA,2013-01-02,2013,2010,2013-01-01,2013,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
29,2023,11,APOSENTADORIA VOLUNTARIA,2013-01-02,2013,2010,2013-01-01,2013,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
32,2023,11,APOSENTADORIA VOLUNTARIA,2014-06-05,2014,2010,2014-06-04,2014,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
34,2023,11,APOSENTADORIA VOLUNTARIA,2014-06-05,2014,2010,2014-06-04,2014,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
36,2023,11,APOSENTADORIA VOLUNTARIA,2014-06-05,2014,2010,2014-06-04,2014,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
411250,2023,11,APOSENTADORIA VOLUNTARIA,1983-04-06,1983,1980,1983-04-04,1983,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
412141,2023,11,APOSENTADORIA POR INVALIDEZ,1973-01-25,1973,1970,1973-01-11,1973,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
413399,2023,11,APOSENTADORIA VOLUNTARIA,1997-08-08,1997,1990,1997-08-01,1997,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0
414344,2023,11,APOSENTADORIA VOLUNTARIA,1998-12-02,1998,1990,1998-06-26,1998,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,0


In [28]:
menor_zero = df[df['tempo_contribuicao_servico_publico']< 0] #242 linhas
menor_zero 

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total,tempo_contribuicao_servico_publico
1156,2023,11,APOSENTADORIA VOLUNTARIA,1983-04-06,1983,1980,1991-01-02,1991,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-8
25447,2023,11,APOSENTADORIA VOLUNTARIA,1995-05-09,1995,1990,2000-04-01,2000,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-5
27835,2023,11,APOSENTADORIA POR INVALIDEZ,1966-12-01,1966,1960,1993-11-01,1993,"Ministério da Ciência, Tecnologia, Inovações e...",REGIME JURIDICO UNICO,1,-27
27837,2023,11,APOSENTADORIA VOLUNTARIA,1979-05-28,1979,1970,1980-04-23,1980,"Ministério da Ciência, Tecnologia, Inovações e...",REGIME JURIDICO UNICO,1,-1
27854,2023,11,APOSENTADORIA POR INVALIDEZ,1978-10-16,1978,1970,1980-04-01,1980,"Ministério da Ciência, Tecnologia, Inovações e...",REGIME JURIDICO UNICO,1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...
412537,2023,11,APOSENTADORIA VOLUNTARIA,1992-07-31,1992,1990,1997-07-01,1997,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-5
412614,2023,11,APOSENTADORIA VOLUNTARIA,1969-04-02,1969,1960,1991-01-31,1991,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-22
413024,2023,11,APOSENTADORIA VOLUNTARIA,1996-06-14,1996,1990,1997-07-01,1997,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-1
413216,2023,11,APOSENTADORIA VOLUNTARIA,1989-06-23,1989,1980,1991-01-31,1991,MIN GESTAO E INOV EM SERV PUBLICOS,REGIME JURIDICO UNICO,1,-2


In [29]:
df = df[df['tempo_contribuicao_servico_publico'] > 0]
df['tempo_contribuicao_servico_publico'].unique()

array([26, 36, 12, 21, 25, 15, 22, 41, 19, 18, 24, 32, 34, 11, 30, 38, 17,
       20, 31, 40, 28, 23, 27,  2, 14, 33, 16, 39, 29, 46, 55, 67, 60, 48,
       56, 57, 42,  5, 37, 43, 13, 44, 45,  1,  7,  3,  4,  8, 35, 50, 47,
       54, 49,  6, 52, 10,  9, 53, 66, 58, 51, 59, 61, 65, 62, 64, 63])

In [30]:
df['decada_aposentadoria'] = df['decada_aposentadoria'].astype(int)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\2506975054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decada_aposentadoria'] = df['decada_aposentadoria'].astype(int)


In [31]:
df['decada_aposentadoria'].unique()

array([2000, 2010, 1980, 1990, 2020, 1970, 1960, 1950])

## Tratamento final

In [32]:
df['decada_aposentadoria'].dtype

dtype('int32')

In [33]:
def renomear(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INVALIDEZ\b", "Aposentadoria por invalidez", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA VOLUNTARIA\b", "Aposentadoria voluntária", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA COMPULSORIA\b", "Aposentadoria compulsória", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INCAPACIDADE\b", "Aposentadoria por incapacidade", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bOUTROS\b", "Outros", var_aposentadoria)
    
    return var_aposentadoria

In [34]:
df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear) 

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\2971334208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear)


In [35]:
df['tipo_aposentadoria'].unique()

array(['Aposentadoria por invalidez', 'Aposentadoria voluntária',
       'Aposentadoria compulsória', 'Aposentadoria por incapacidade',
       'Outros'], dtype=object)

In [36]:
df['org_lotacao'] = df['org_lotacao'].str.title()

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\2304693947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['org_lotacao'] = df['org_lotacao'].str.title()


In [37]:
df['org_lotacao'].unique()

array(['Min Gestao E Inov Em Serv Publicos', 'Ministerio Da Cultura',
       'Ministério Da Saúde', 'Ministério Da Defesa',
       'Ministério Da Educação', 'Ministerio Dos Povos Indigenas',
       'Presidência Da República', 'Ministério De Minas E Energia',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministerio Dos Transportes', 'Ministerio Da Previdencia Social',
       'Ministerio Do Planejamento E Orcamento', 'Ministerio Da Fazenda',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Min Da Integ E Do Desenv Regional',
       'Min Do Desenv Agr E Agric Familiar',
       'Ministerio Do Meio Ambiente', 'Ministério Do Turismo',
       'Ministerio Do Trabalho E Emprego',
       'Ministerio De Portos E Aeroportos',
       'Min Desenvolv Ind Comercio E Servicos',
       'Controladoria-Geral Da União', 'Ministerio Das Comunicacoes',
       '

In [38]:
def renomear2(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bMin Gestao E Inov Em Serv Publicos\b", "Ministério Gestão E Inovação Em Serviços Públicos", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Dos Povos Indigenas\b", "Ministário dos Povos Indígenas", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Da Previdencia Social\b", "Ministério Da Previdência Social", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Planejamento E Orcamento\b", "Ministério Do Planejamento E Orçamento", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Da Integ E Do Desenv Regional\b", "Ministério da Integração E Do Desenvolvimento Regional", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Trabalho E Previdencia\b", "Ministério do Trabalho E Previdência", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Do Desenv Agr E Agric Familiar\b", "Ministério do Desenvolvimento Agrário E Agricultura Familiar", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Desenvolv Ind Comercio E Servicos\b", "Ministério do Desenvolvimento, Indústria, Comércio E Serviços", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bComunicacoes\b", "Comunicações", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio\b", "Ministério", var_aposentadoria)
    return var_aposentadoria

In [39]:
df['org_lotacao'] = df['org_lotacao'].apply(renomear2)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\3127125097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['org_lotacao'] = df['org_lotacao'].apply(renomear2)


In [40]:
def renomear3(var_regime):
    var_regime = re.sub(r"\bREGIME JURIDICO UNICO\b", "Regime Jurídico Único", var_regime)
    return var_regime

In [41]:
df['regime_juridico'] = df['regime_juridico'].apply(renomear3)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\283232767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['regime_juridico'] = df['regime_juridico'].apply(renomear3)


In [42]:
ministerios = {'Ministério Gestão E Inovação Em Serviços Públicos': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Da Cultura': 'Cultura, Turismo e Esportes',
                'Ministério Da Saúde': 'Saúde',
                'Ministério Da Defesa': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Da Educação': 'Educação e Ciência e Tecnologia',
                'Ministário dos Povos Indígenas': 'Desenvolvimento Social e Direitos Humanos',
                'Presidência Da República': 'Presidência da República',
                'Ministério De Minas E Energia': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério Da Justiça E Segurança Pública': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Da Ciência, Tecnologia, Inovações E Comunicações': 'Educação e Ciência e Tecnologia',
                'Ministério Dos Transportes': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério Da Previdência Social': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Do Planejamento E Orçamento': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Da Fazenda': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério Das Relações Exteriores': 'Relações Exteriores',
                'Ministério Da Agricultura, Pecuária E Abastecimento': 'Meio ambiente, Agricultura e Agropecuária',
                'Ministério da Integração E Do Desenvolvimento Regional': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério do Desenvolvimento Agrário E Agricultura Familiar': 'Meio ambiente, Agricultura e Agropecuária',
                'Ministério Do Meio Ambiente': 'Meio ambiente, Agricultura e Agropecuária',
                'Ministério Do Turismo': 'Cultura, Turismo e Esportes',
                'Ministério Do Trabalho E Emprego': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Ministério De Portos E Aeroportos': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério do Desenvolvimento, Indústria, Comércio E Serviços': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Controladoria-Geral Da União': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Das Comunicações': 'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
                'Ministério do Trabalho E Previdência': 'Economia, Finanças, Planejamento, Previdência e Gestão',
                'Defensoria Pública Da União': 'Justiça, Segurança e Órgãos de Controle e Defesa',
                'Ministério Da Economia': 'Economia, Finanças, Planejamento, Previdência e Gestão'
}


In [43]:
def categorizando(x):
    if x in ministerios:
        return ministerios[x]


In [44]:
df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_2012\3148416238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)


In [45]:
df['categoria_ministerios'].unique()

array(['Economia, Finanças, Planejamento, Previdência e Gestão',
       'Cultura, Turismo e Esportes', 'Saúde',
       'Justiça, Segurança e Órgãos de Controle e Defesa',
       'Educação e Ciência e Tecnologia',
       'Desenvolvimento Social e Direitos Humanos',
       'Presidência da República',
       'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
       'Relações Exteriores', 'Meio ambiente, Agricultura e Agropecuária'],
      dtype=object)

In [46]:
df.columns

Index(['ano', 'mes', 'tipo_aposentadoria', 'data_aposentadoria',
       'ano_aposentadoria', 'decada_aposentadoria',
       'data_diploma_ingresso_servicopublico', 'ano_ingresso', 'org_lotacao',
       'regime_juridico', 'total', 'tempo_contribuicao_servico_publico',
       'categoria_ministerios'],
      dtype='object')

In [47]:
df

,ano,mes,tipo_aposentadoria,data_aposentadoria,ano_aposentadoria,decada_aposentadoria,data_diploma_ingresso_servicopublico,ano_ingresso,org_lotacao,regime_juridico,total,tempo_contribuicao_servico_publico,categoria_ministerios
0,2023,11,Aposentadoria por invalidez,2008-04-30,2008,2000,1982-08-12,1982,Ministério Gestão E Inovação Em Serviços Públicos,Regime Jurídico Único,1,26,"Economia, Finanças, Planejamento, Previdência ..."
1,2023,11,Aposentadoria voluntária,2017-04-07,2017,2010,1981-08-01,1981,Ministério Da Cultura,Regime Jurídico Único,1,36,"Cultura, Turismo e Esportes"
2,2023,11,Aposentadoria voluntária,2002-04-12,2002,2000,1990-03-15,1990,Ministério Gestão E Inovação Em Serviços Públicos,Regime Jurídico Único,1,12,"Economia, Finanças, Planejamento, Previdência ..."
3,2023,11,Aposentadoria voluntária,2011-05-12,2011,2010,1990-10-01,1990,Ministério Gestão E Inovação Em Serviços Públicos,Regime Jurídico Único,1,21,"Economia, Finanças, Planejamento, Previdência ..."
4,2023,11,Aposentadoria voluntária,2002-06-14,2002,2000,1977-02-15,1977,Ministério Gestão E Inovação Em Serviços Públicos,Regime Jurídico Único,1,25,"Economia, Finanças, Planejamento, Previdência ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418496,2023,11,Aposentadoria voluntária,2009-09-30,2009,2000,1979-09-01,1979,Ministério Da Defesa,Regime Jurídico Único,1,30,"Justiça, Segurança e Órgãos de Controle e Defesa"
418497,2023,11,Aposentadoria voluntária,2012-04-02,2012,2010,1980-09-17,1980,Ministério Da Defesa,Regime Jurídico Único,1,32,"Justiça, Segurança e Órgãos de Controle e Defesa"
418498,2023,11,Aposentadoria voluntária,1994-09-06,1994,1990,1984-04-01,1984,Ministério Da Defesa,Regime Jurídico Único,1,10,"Justiça, Segurança e Órgãos de Controle e Defesa"
418499,2023,11,Aposentadoria por invalidez,2009-09-15,2009,2000,1977-09-01,1977,Ministério Da Defesa,Regime Jurídico Único,1,32,"Justiça, Segurança e Órgãos de Controle e Defesa"


In [128]:
df1 = pd.pivot_table(df, values='tempo_contribuicao_servico_publico', index=['tipo_aposentadoria', 'decada_aposentadoria', 'categoria_ministerios'], aggfunc=['mean','count']).reset_index()


In [164]:
df1 # Recorte do df original para fazer essa tabela dinâmica

,tipo_aposentadoria,decada_aposentadoria,categoria_ministerios,media_contribuicao,quantidade_vinculos
0,Aposentadoria compulsória,1960,"Justiça, Segurança e Órgãos de Controle e Defesa",2.0,1
1,Aposentadoria compulsória,1970,Educação e Ciência e Tecnologia,16.0,1
2,Aposentadoria compulsória,1970,"Justiça, Segurança e Órgãos de Controle e Defesa",4.8,30
3,Aposentadoria compulsória,1980,Educação e Ciência e Tecnologia,16.0,2
4,Aposentadoria compulsória,1980,"Justiça, Segurança e Órgãos de Controle e Defesa",23.5,16
...,...,...,...,...,...
200,Outros,2000,Educação e Ciência e Tecnologia,34.5,8
201,Outros,2000,"Justiça, Segurança e Órgãos de Controle e Defesa",36.7,3
202,Outros,2000,Saúde,21.0,3
203,Outros,2010,"Economia, Finanças, Planejamento, Previdência ...",30.0,1


In [130]:
df1.columns = df1.columns.droplevel(level=1) 

In [131]:
df1 = df1.rename(columns={'mean':'media_contribuicao',
                          'count':'quantidade_vinculos'})

In [171]:
df1['media_contribuicao'].unique()

array([ 2. , 16. ,  4.8, 23.5, 19.5, 16.5, 24. , 21.2, 21. , 23.6, 17. ,
       23. , 36.9, 22. , 22.5, 22.9, 24.3, 24.1, 28.2, 24.8, 27.3, 42.4,
       26.9, 25.6, 28.9, 25.5, 21.1, 15.9, 25.9, 26. , 24.2, 30.3, 25.4,
       32.7, 22.4, 19.6, 39.3, 33.8, 13. ,  1. , 18.7, 22.7, 17.9, 16.4,
       27.2, 20.1, 29.9, 22.1,  6.2,  5. ,  9.4,  8.8,  9.7, 13.5, 17.6,
       20. , 10.5, 15.5, 12. , 21.8, 19.7, 17.8, 35. , 23.2, 26.2, 20.7,
       25.8, 12.9, 16.8, 18.1, 18.3, 20.8, 19.4, 21.7, 21.6, 19.8, 20.6,
       25. , 22.3, 24.7, 30.7, 25.7, 22.6, 30. , 19.2, 20.3, 17.7, 26.4,
        5.5, 15.4,  9.8, 14.4,  8. , 15. , 11.4, 20.2, 23.4, 29.5, 26.3,
       28.3, 28.6, 27. , 18.4, 23.1, 22.8, 28.7, 26.7, 35.5, 27.4, 32.6,
       34.7, 32.4, 31.2, 34.8, 33.9, 34.5, 40.5, 33.7, 36.8, 37.2, 36. ,
       38.5, 34.3, 38.6, 42.8, 38.2, 26.6, 29. , 14. , 23.7, 28. , 36.7])

In [170]:
df1['media_contribuicao'] = df1['media_contribuicao'].round(1) # Limitando as casas decimais

In [139]:
df1.columns

Index(['tipo_aposentadoria', 'decada_aposentadoria', 'categoria_ministerios',
       'media_contribuicao', 'quantidade_vinculos'],
      dtype='object')

In [168]:
df1['media_contribuicao'].describe()

count    205.000000
mean      22.909268
std        8.079954
min        1.000000
25%       18.400000
50%       22.900000
75%       27.000000
max       42.800000
Name: media_contribuicao, dtype: float64

In [141]:
df1['quantidade_vinculos'].describe()

count      205.000000
mean      2000.629268
std       6672.592028
min          1.000000
25%          8.000000
50%         76.000000
75%        533.000000
max      50654.000000
Name: quantidade_vinculos, dtype: float64

In [167]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [143]:
df1.columns

Index(['tipo_aposentadoria', 'decada_aposentadoria', 'categoria_ministerios',
       'media_contribuicao', 'quantidade_vinculos'],
      dtype='object')

In [103]:
df1['decada_aposentadoria'].unique()

array([1960, 1970, 1980, 1990, 2000, 2010, 2020, 1950], dtype=int64)

In [146]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tipo_aposentadoria     205 non-null    object 
 1   decada_aposentadoria   205 non-null    int64  
 2   categoria_ministerios  205 non-null    object 
 3   media_contribuicao     205 non-null    float64
 4   quantidade_vinculos    205 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 8.1+ KB


In [172]:
schema=[bigquery.SchemaField('tipo_aposentadoria','STRING',description='Tipo de aposentadoria'),
        bigquery.SchemaField('decada_aposentadoria','INTEGER',description='Década da aposentadoria'),       
        bigquery.SchemaField('categoria_ministerios','STRING',description='Categorias dos ministérios'),
        bigquery.SchemaField('media_contribuicao','FLOAT',description='Média do tempo de aposentadorias agrupadas por setor, década da aposentadoria e tipo da aposentadoria'),
        bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade total de aposentadorias agrupadas por setor, década da aposentadoria e tipo da aposentadoria')
        ]


In [173]:
table_ref = dataset_ref.table('SIAPE_ingresso_aposentadoria_media') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=9eb141ae-1867-4b66-85f6-ff3d5cd27ea6>